In [105]:
%matplotlib inline

In [106]:
import sys, os, requests, re
from pathlib import Path
import pathlib
streetview_path = Path(os.path.abspath('')).parent.absolute()

if (not (streetview_path in sys.path)) :
    sys.path.append(str(streetview_path))

In [107]:
from xml.dom import minidom
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import numpy as np

import streetview

### Overpass

In [59]:
import overpy
import time
import folium
import numpy as np
import geopandas as gpd

api = overpy.Overpass()

# Effiel Tower (France)
latitude = 48.8583701
longitude = 2.2944813

# Hoan Kiem Lake (Vietname)
# lat = 21.01931
# lng = 105.830341

radius = 200

In [3]:
# https://wiki.openstreetmap.org/wiki/Key:highway#Roads
# phong an

highway_allow = ["motorway","trunk","primary","secondary","tertiary","unclassified",
                 "residential"," motorway_link","trunk_link","primary_link",
                 "secondary_link","tertiary_link","pedestrian"]
allow_list = "|".join(highway_allow)
allow_list

'motorway|trunk|primary|secondary|tertiary|unclassified|residential| motorway_link|trunk_link|primary_link|secondary_link|tertiary_link|pedestrian'

In [70]:
# Create a map centered on the specified latitude and longitude
m = folium.Map(location=[latitude, longitude], zoom_start=14)

# Query the Overpass API for ways within the specified radius
api = overpy.Overpass()
# result = api.query(f"""
#     way(around:{radius},{latitude},{longitude})[highway];
#     out;
# """)
result = api.query(f"""
    way(around:{radius},{latitude},{longitude})[highway~"^({allow_list})$"];
    out;
""")

In [71]:
waypoint = set()

for way in result.get_ways():
    while True:
        try:
            
            polyline = list()
            
            nodes = way.get_nodes(resolve_missing=True)
            for node in way.nodes:
                polyline.append( (node.lat, node.lon) )
                waypoint.add( (node.lat, node.lon) )
            
            folium.PolyLine(polyline).add_to(m)
        except OverpassTooManyRequests:
            time.sleep(1)
            print('Retrying...')
            continue
        except OverpassGatewayTimeout:
            time.sleep(10)
            print('OverpassGatewayTimeout, retrying...')
            continue
        break

In [91]:
waypoint

{(Decimal('48.8524195'), Decimal('2.2849271')),
 (Decimal('48.8524644'), Decimal('2.2849363')),
 (Decimal('48.8530741'), Decimal('2.2857879')),
 (Decimal('48.8539830'), Decimal('2.2870620')),
 (Decimal('48.8546113'), Decimal('2.2879390')),
 (Decimal('48.8547728'), Decimal('2.2881939')),
 (Decimal('48.8550375'), Decimal('2.2885130')),
 (Decimal('48.8550383'), Decimal('2.2961229')),
 (Decimal('48.8550894'), Decimal('2.2960410')),
 (Decimal('48.8551120'), Decimal('2.2960048')),
 (Decimal('48.8552713'), Decimal('2.2957598')),
 (Decimal('48.8553047'), Decimal('2.2887438')),
 (Decimal('48.8553507'), Decimal('2.2956361')),
 (Decimal('48.8553800'), Decimal('2.2966295')),
 (Decimal('48.8554283'), Decimal('2.2965469')),
 (Decimal('48.8557335'), Decimal('2.2890386')),
 (Decimal('48.8558260'), Decimal('2.2959411')),
 (Decimal('48.8559327'), Decimal('2.2947290')),
 (Decimal('48.8559842'), Decimal('2.2946486')),
 (Decimal('48.8560123'), Decimal('2.2973257')),
 (Decimal('48.8560344'), Decimal('2.2945

In [72]:
m

In [ ]:
def expand_lat_lon(lat_lon: tuple, lat_lon_before: tuple) -> list:
    
    """Expand latitude longitude to get more images between two points"""
    
    lat, lon = lat_lon
    lat_before, lon_before = lat_lon_before
    diff_lat = lat - lat_before
    diff_lon = lon - lon_before
    step_size = 3e-5
    steps_lat = abs(diff_lat) / 1e-5
    steps_lon = abs(diff_lon) / 1e-5
    factor_lat = step_size if diff_lat > 0 else -step_size
    factor_lon = step_size if diff_lon > 0 else -step_size
    jumps = round(max([steps_lat, steps_lon]) / min([steps_lat, steps_lon]))
    expanded = []

    for i in range(int(max([steps_lat, steps_lon]))):

        if not (i % jumps):
            # Find the smaller one
            if steps_lat <= steps_lon:
                lat += factor_lat
            else:
                lon += factor_lon

        # Find the bigger one
        if steps_lat >= steps_lon:
            lat += factor_lat
        else:
            lon += factor_lon
        
        expanded.append((lat, lon))
    
    return expanded


def download_images(coords: list, 
                    show_img: bool=True, 
                    img_dir: str='', 
                    video_dir: str='') -> None:
    """Function to download panorama images from coordinates list"""
    if img_dir:
        img_cnt = 0
        check_path(img_dir)
        
    if video_dir:
        check_path(video_dir)
    
    lat_before = None
    lon_before = None
    pan_before = None
    
    for lat, lon in coords:
        
        if lat_before is not None:
            expanded_lat_lon = expand_lat_lon((lat, lon), (lat_before, lon_before))
        
        if 'expanded_lat_lon' in locals():
            for e_lat, e_lon in expanded_lat_lon:
                print(f"Searching for lat={e_lat}, lon={e_lon}")
                panoids = streetview.panoids(lat=e_lat, lon=e_lon)
                try:
                    panoid = panoids[0]['panoid']
                except IndexError:
                    continue
                panorama = streetview.download_panorama_v3(panoid, zoom=3, disp=False)
                # Don't save the same image twice
                if pan_before is not None:
                    if np.array_equal(panorama, pan_before):
                        continue
                        
                pan_before = panorama
                
                if show_img:
                    plt.figure(figsize=(15, 15))
                    plt.imshow(panorama)
                    plt.axis('off')
                    plt.show()

                # Save img
                if img_dir:
                    # file name format: img_number_latitude_longitude.png
                    plt.imsave(f"{img_dir}/img_{img_cnt}_{lat}_{lon}.jpg", panorama)
                    img_cnt += 1
        
        # Skip first frame
        lat_before = lat
        lon_before = lon
        
    if 'out' in locals():
        out.release()

In [ ]:
coords = get_lat_long(gpx_path)

In [ ]:
download_images(coords, show_img=True, img_dir='./images/', video_dir='./video/')

In [ ]:
panoids = streetview.panoids(lat=40.75388056, lon=-73.99697222)
panoid = panoids[0]['panoid']
panorama = streetview.download_panorama_v3(panoid, zoom=2, disp=False)
plt.imshow(panorama)

In [ ]:
panoid = "ChAzMTM1YWI3NzhlYTEwYmI3"
panorama = streetview.download_panorama_v3(panoid, zoom=2, disp=False)
panorama

### Refacor sv-dlp

In [25]:
from PIL import Image

In [27]:
metadata = None

In [42]:
class MetadataStructure:
    dict_instance = list()
    
    def __init__(self, service=None, pano_id=None, lat=None, lng=None, date=None, size=None, max_zoom=None, misc={}, timeline={}, linked_panos={}):
        self.pano_id = pano_id
        self.lat = lat
        self.lng = lng
        self.date = date
        self.size = size
        self.max_zoom = max_zoom
        self.timeline = timeline
        self.dict_instance.append(self)
        
    def __repr__(self):
        return (
                f"{self.__class__.__name__}("
                f"pano_id={self.pano_id}, "
                f"lat={self.lat}, "
                f"lng={self.lng}, "
                f"date={self.date}, "
                f"size={self.size}, "
                f"max_zoom={self.max_zoom}, "
                f"timeline={self.timeline}, "
        )
        
    @classmethod
    def dict(cls):
        for instance in cls.dict_instance:
            return instance.__dict__

In [44]:
def get_metadata(pano_id=None, lat=None, lng=None, get_linked_panos=False) -> MetadataStructure:
        """
        Calls allocated service's `get_metadata()` function to obtain metadata
        with given input, and store it to class and variable.
        
        Metadata is returned in a `MetadataStructure` object, providing
        the developer a more structured and organized way to handle 
        metadata information with various attributes.
        Additionally, the `.dict()` method returns the attributes of 
        each instance of the MetadataStructure class in the form of a 
        dictionary, allowing for easy access and manipulation of the 
        metadata information.
        
        sv_dlp's metadata structure is designed with compatibility in mind, 
        allowing developers to tinker with it no matter the service picked. 
        An example of the returned metadata is the one below:
        ```python
        metadata = MetadataStructure(
            service=service, 
            pano_id=pano_id, 
            lat=lat, 
            lng=lng, 
            date=datetime.datetime(), 
            size=image_size, 
            max_zoom=max_zoom, 
            timeline=[{'pano_id': 'pano_id', 'date': datetime.datetime()}], 
            linked_panos={{'pano_id': pano_id, 'lat': lat, 'lng': lng, 'date': datetime.datetime()}}, 
            misc={}
        )
        ```
        
        Additionally, the developer has the option to access the metadata 
        in dictionary form by calling the `.dict()` method. An example is:
        ```python
        metadata = {
            "service": service,
            "pano_id": pano_id,
            "lat": lat,
            "lng": lng,
            "date": datetime.datetime(),
            "size": image_size,
            "max_zoom": max_zoom,
            "misc": { # Only use with exclusive service features
                "is_trekker": len(json[1][0][5][0][3][0][0][2]) > 3,
                "gen": gen,
            },
            "timeline": {
                [{'pano_id'}: pano_id, "date": date}],
                [{'pano_id'}: pano_id, "date": date}],
                [{'pano_id'}: pano_id, "date": date}],
                # and so on...
            }
            "linked_panos": {
                [{'pano_id'}: pano_id, "date": date, "lat": lat, "lng" lng}],
                [{'pano_id'}: pano_id, "date": date, "lat": lat, "lng" lng}],
                [{'pano_id'}: pano_id, "date": date, "lat": lat, "lng" lng}],
                # and so on... 
                # only added if get_linked_panos is true
            },
        }
        ```
        
        Parameters
        ----------
        str:    pano_id
            Panorama ID - Might not work with some services
        float:  lat
            Latitude
        float:  lng
            Longitude
        bool:   get_linked_panos
            Sets if linked panos should be returned or not

        Returns
        ----------
        MetadataStructure:  metadata
            Metadata of given input
        MetadataStructure:  self.metadata
            Stores metadata in class
        func:               .dict()
            Function inside object that translates `MetadataStructure` object to dictionary
        """
        md = get_metadata(pano_id=pano_id, lat=lat, lng=lng)
        self.metadata = md
        return md

In [84]:
def _get_pano_from_coords(lat, lng, radius=500) -> dict:
    """
    Returns closest Google panorama ID to given parsed coordinates.
    """
    try:
        url = streetview.urls._build_metadata_url(lat=lat, lng=lng, mode="SingleImageSearch", radius=radius)
        json = requests.get(url).text
        if "Search returned no images." in json:
            print("[google]: Finding nearest panorama via satellite zoom...")
            url = urls._build_metadata_url(lat=lat, lng=lng, mode="SatelliteZoom")
            json = requests.get(url).text
            data = j.loads(json[4:])
            pano = data[1][1][0][0][0][1]
        else:
            data = re.findall(r'\[[0-9],"(.+?)"].+?,\[\[null,null,(.+?),(.+?)\]', json)
            pano = data[0][0]
    except TypeError:
        raise sv_dlp.services.NoPanoIDAvailable
    # pans = re.findall(r"\[[0-9],"(.+?)"].+?,\[\[null,null,(.+?),(.+?)\]", json)
    return pano

In [85]:
_get_pano_from_coords(lat, lng)

_xdc_._b42go0


'PX1BkjvmzSwoov98jBN5HQ'

In [75]:
def _build_tile_arr(panoid = None, zoom=3, alternate=False):

    coord = list(itertools.product(range(imgx), range(13)))

    if alternate:
        image_url = 'https://lh3.ggpht.com/p/{}=x{}-y{}-z{}'
        tiles = [(x, y, "%s_%dx%d.jpg" % (panoid, x, y), image_url.format(panoid, x, y, zoom)) for x, y in coord]
    else:
        image_url = 'https://streetviewpixels-pa.googleapis.com/v1/tile?cb_client=maps_sv.tactile&panoid={}&zoom={}&x={}&y={}'
        tiles = [(x, y, "%s_%dx%d.jpg" % (panoid, x, y), image_url.format(panoid, zoom, x, y)) for x, y in coord]
    
    return tiles

In [51]:
def download_panorama(pano_id=None, lat=None, lng=None, zoom=3) -> Image:
        """
        Obtains Tile URLs List from a given Panorama ID/Coordinate with a
        specified zoom, downloads each row in a multithreaded way and stitches them.
        
        If self.metadata is not allocated (or does not match with given input),
        `get_metadata` is automatically called.
        
        Parameters
        ----------
        str:    pano_id
            Panorama ID - Might not work with some services
        float:  lat
            Latitude
        float:  lng
            Longitude

        Returns
        ----------
        Image:  img
            Stitched Panorama Image in PIL.Image format
        Image:  tile_imgs
            List of Tile Images where each element is stored
            in a PIL.Image format
        Tuple:  tuple
            A tuple is returned if only one
            variable is assigned
        """

        if metadata == None:
            print(f"Obtainin Metadata...")
            if pano_id != None:
                get_metadata(pano_id=pano_id)
            elif lat and lng != None:
                get_metadata(lat=lat, lng=lng)

        if zoom == -1:
            zoom = self.metadata.max_zoom / 2
            zoom = round(zoom + .1)

        print(f"[{self.service_str}]: Building Tile URLs...")
        tile_arr = streetview.urls._build_tile_arr(self.metadata, zoom)
        img, tiles_imgs = download.panorama(tile_arr, self.metadata)
        return img, tiles_imgs


In [54]:
pano = "fzJzOcJLZPq-_QPBJzl5Dg"
lat=6.603079535799973
lng=-73.99819681137278
print(download_panorama(pano))

Obtainin Metadata...


RecursionError: maximum recursion depth exceeded

In [88]:
streetview.metadata._get_panoid_from_coord(latitude, longitude)

AttributeError: module 'streetview' has no attribute 'metadata'

In [37]:
size=[[[[256, 512]], [[512, 1024]], [[1024, 2048]], [[2048, 4096]], [[4096, 8192]], [[8192, 16384]]], [512, 512]]
size[0]

[[[256, 512]],
 [[512, 1024]],
 [[1024, 2048]],
 [[2048, 4096]],
 [[4096, 8192]],
 [[8192, 16384]]]

In [39]:
size[0][3][0][0]

2048

In [40]:
size[1][0]

512

In [41]:
2048 // 512

4

In [102]:
coords = [('lYCBXiB9avkEghp8rW5oWQ', '48.85816212295677', '2.292259828699704'), ('lYCBXiB9avkEghp8rW5oWQ', '48.85816212295677', '2.292259828699704'), ('l2OhUYnXVg8f7C5UkUQkNA', '48.85822687621564', '2.292353127940943'), ('HFF3RXWXIFDV4OOZLX01SA', '48.85802747002063', '2.291747792477793'), ('eXdEqtAxrMpYuxNI24tNZQ', '48.85809096248042', '2.291842700701102'), ('Ftm16xhJWS-3PI_r5rni1g', '48.85815608461488', '2.291940653874237'), ('sDHO_lpzniHaL4wtiKSE7w', '48.85822141986709', '2.292039481572949'), ('9nTL9e82X5RunTcebTgcpg', '48.85828708903684', '2.292136966175553'), ('bbpw-bohPRi_UkfSG-vakQ', '48.85835384554021', '2.292233853890011'), ('jcz48RYOB0uiP78T3gguqg', '48.8584206037313', '2.292330656695505'), ('wdDUAqyBgMi1i0WfTURYyw', '48.85848751870361', '2.292427301451673'), ('wIITz3z19HbKjNKxrwjptA', '48.85855458536836', '2.292524044758148'), ('IEFIAvhy7h2mo8g5H12zrQ', '48.85787585922505', '2.291823612576694'), ('kOLQGAbuwsjeEM5iYbJZVg', '48.85793857467075', '2.291922826937287'), ('De2GPkypnbwX4Zs_IUrcuQ', '48.85800267065402', '2.292022153437689'), ('OsOc9voVYxH7Pp5DoUFoDA', '48.85806612213646', '2.292119115456417'), ('wDjMoIf8FpeO2jEh0qH3MQ', '48.85829171804122', '2.292445835232821'), ('7pgZYbjv7-2FvOXmSkzW3w', '48.85835610476156', '2.292538249320226'), ('HBCNqjSjwyR2E04u091FRA', '48.85842083424689', '2.292632747762904'), ('fXXItxYouTWmx8WENgGjwg', '48.85848594382139', '2.292727448851989'), ('GqPdbBjEnyN3bBeeqA699Q', '48.85809311470431', '2.291734162114328'), ('dAXROEo7VfAkQ-2BInCRfQ', '48.85815860648437', '2.29182897496927'), ('WKvXlX_S7cuNI0F29NyL9g', '48.85822506310517', '2.291925494060548'), ('BVS8Nyybysm8u0PLETpajw', '48.8582582914123', '2.291973753838193'), ('Gh9dLefFk9KPMZVrxF61Jg', '48.85832557134534', '2.292071368395626'), ('8RAUleaFEXtyUp0IVkX_Cw', '48.85839330128138', '2.292169533957144'), ('rOmWH2CFvml3Ij1R0oNl7A', '48.85846127317851', '2.292267120020071'), ('AxTOQJ1mX_zxyIqwSEkljg', '48.85852882559302', '2.292362616747308'), ('RKvn8Oq4f3Z3Al1giCV08g', '48.85803732420326', '2.29167311535068'), ('zwPsiYDrC77k7zN14HkDMQ', '48.85796525219678', '2.291650589110629'), ('-bHT3X3I2vgvo53PaLgO6A', '48.85824484698456', '2.292617659720833'), ('7Ow_pmNUr-NhLNqgVsP3RA', '48.85836203128781', '2.29278286989657'), ('CIkJczbGY-TdDMc8_9kQFg', '48.85792079420321', '2.292623410952928'), ('DzlYMMbvfuY597tWO8oG4g', '48.85831542158429', '2.292716412183751'), ('ErcRFcj-Md1KP1qeKfhfMA', '48.85807905624706', '2.292440714931843'), ('L7yrs702WqHUhG55JxXrTQ', '48.85802649219269', '2.292504057091264'), ('PXu76ESfJd3-N6wklaa-QA', '48.85812476353783', '2.292454426678148'), ('dCKrm24vhxgBirMrt_RbHg', '48.85784031298723', '2.292707977589089'), ('xGD-eZRNG-_uX2fHMJ4GFg', '48.85797249962298', '2.292558720607019'), ('ye8NgFrlUjM6b9Zff_2wMw', '48.85817120770319', '2.292518712271686'), ('5W6yfSbpOFfDYsj_sIZc2A', '48.85816882524027', '2.292266624000511'), ('2qNadrY5p2jr_n_GtcZe0A', '48.85816694269232', '2.292256962665467'), ('-Ruf3YeXqKamJUUNe0A3zQ', '48.85819956709004', '2.292246576683187'), ('Mzpr0dHjParoiPlSMg21kA', '48.85815437270938', '2.292255518803615'), ('xn9d25WhfjRAY9janGnDaQ', '48.85817983772812', '2.292307863773223'), ('KotqdHqn93uaXU1RaYTYUw', '48.85817927334006', '2.292305333775035'), ('l8VkT48s3WPDD6Mf9_BLqA', '48.85817426364947', '2.292263550941798'), ('SvM4nEmLBpLzxAmmQsNf-A', '48.85819923898339', '2.292201145660275'), ('U7XVbucDOiVn91BCxFQTAA', '48.85816071597404', '2.292268795929879'), ('Nke_uFcEhw3rtSt-_wb7BA', '48.85818169959941', '2.292260467924112'), ('txD7N_0i2gTEQHhWh9DIsQ', '48.85812986137158', '2.292213188591407')]

In [104]:
center_lat, center_lon = float(coords[0][1]), float(coords[0][2])

# Create a Folium map centered on the first point
m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

# Add a marker for each coordinate to the map
first_lat, first_lon = float(coords[0][1]), float(coords[0][2])
first_marker = folium.Marker(location=[first_lat, first_lon], color='red')
first_marker.add_to(m)

# Create a list of coordinate pairs for the line
line_coords = [(float(coord[1]), float(coord[2])) for coord in coords[1:]]

# Create a PolyLine object and add it to the map
line = folium.PolyLine(locations=line_coords)
line.add_to(m)

# Display the map
m

In [44]:
raw_md = [[0], [[[1], [2, '5hjX-Yarr0TwxzhXu_FkNg'], [2, 2, [8192, 16384], [[[[256, 512]], [[512, 1024]], [[1024, 2048]], [[2048, 4096]], [[4096, 8192]], [[8192, 16384]]], [512, 512]], None, None, None, None, None, '5hjX-Yarr0TwxzhXu_FkNg'], [None, None, [['300 Port de la Bourdonnais', 'fr'], ['Paris, \\xc3\\x8ele-de-France', 'en']]], [[[['\\xc2\\xa9 2023 Google']]]], [[[1], [[None, None, 48.86128119507111, 2.295314966617388], [29.33272171020508], [48.66464614868164, 90.3116226196289, 356.5496826171875], None, 'FR'], None, [[[[2, '5hjX-Yarr0TwxzhXu_FkNg'], None, [[None, None, 48.86128119507111, 2.295314966617388], [29.33272171020508], [48.66464614868164, 90.3116226196289, 356.5496826171875]]], [[2, 'P8A5ljT-kyajVoNMihoZ-g'], None, [[None, None, 48.86134331831732, 2.295420841998263], [29.28767013549805], [48.59828567504883, 89.77833557128906, 356.4241333007812]], [None, None, [['Port de la Bourdonnais', 'fr']]]], [[2, 'blFcaeu-LX1HwUucDIkHJQ'], None, [[None, None, 48.86124063275437, 2.295198840434722], [29.18351936340332], [230.1939849853516, 90.7197036743164, 2.547044515609741]], [None, None, [['Port de la Bourdonnais', 'fr']]]], [[2, '0h_702dU9Mej3nwwxlNDAQ'], None, [[None, None, 48.86113720359652, 2.295463031353794], [33.45352935791016], [44.69409942626953, 86.33415985107422, 0.9682840704917908]]], [[2, 'jZy5z48WevCLiwCOoabVmg'], None, [[None, None, 48.86120294973336, 2.295560545956613], [32.62551498413086], [45.51588439941406, 86.69369506835938, 0.7547216415405273]]], [[2, 'aP4zYYF4ykL45EhQVMjNVg'], None, [[None, None, 48.86126546612085, 2.295662084063208], [31.98527145385742], [48.27001953125, 86.44165802001953, 1.05013370513916]]], [[2, 'z-7yyYS0iOx4QmRLOArifA'], None, [[None, None, 48.86132148320181, 2.295762068709597], [31.38845443725586], [50.4669075012207, 86.64552307128906, 0.7595438957214355]]], [[2, 'k82cOe2RxGWXDh0YSwZYXA'], None, [[None, None, 48.86137684754457, 2.295863971868348], [30.85690116882324], [51.00740814208984, 86.19342803955078, 359.9333190917969]]], [[2, 'GCvaToeTL4M9lDh2bOmSUw'], None, [[None, None, 48.86151408862823, 2.295742522434937], [29.31902694702148], [52.6986083984375, 90.28788757324219, 356.5663757324219]]], [[2, 'iwW2dcVeE9aovfJnUXjU5A'], None, [[None, None, 48.86145976000832, 2.295634876567284], [29.27162170410156], [52.75699615478516, 90.33470916748047, 356.5652465820312]]], [[2, 'V9PAQsS5OhEWYmDLz-Q3Eg'], None, [[None, None, 48.86140426041256, 2.295528164804226], [29.23188018798828], [50.2611198425293, 89.45692443847656, 356.4146423339844]]], [[2, 'SoQca6juzIKq95f9RrbLDg'], None, [[None, None, 48.86117723578832, 2.29508746532672], [29.20486640930176], [228.8867340087891, 90.46295928955078, 2.769518136978149]]], [[2, 'fULE6kljxx4MOp88XpJDdQ'], None, [[None, None, 48.86111442547835, 2.294981365022558], [29.20341300964355], [227.0656127929688, 89.79313659667969, 2.836742401123047]]], [[2, 'jyBgVFPjo2RSzGu5VTZvkA'], None, [[None, None, 48.86099876805216, 2.295253990178788], [34.71953582763672], [44.37556076049805, 87.96726989746094, 0.8977888226509094]]], [[2, 'rAWjOvySz5K0Xy6qUu-Smg'], None, [[None, None, 48.86106956343954, 2.295360308857311], [34.11092376708984], [44.59446716308594, 86.48767852783203, 0.6004412770271301]]], [[2, '9oFMEAvJ3ESjR7xx7ciqAg'], None, [[None, None, 48.86091139205011, 2.295154727835233], [34.97118377685547], [1.862527132034302, 90.19285583496094, 0.5110283493995667]]], [[2, 'E_3Ope-yb7OOGwmzsoEJbA'], None, [[None, None, 48.8610483803414, 2.29487954489913], [29.36871719360352], [224.5025939941406, 89.54876708984375, 3.162627458572388]]], [[2, 'yWjAwN0FXLMHd1A27JKcOA'], None, [[None, None, 48.86129679955308, 2.295314515323192], [29.22339057922363], [52.39143371582031, 90.56421661376953, 355.9833679199219]], [None, None, [['Port de la Bourdonnais', 'fr']]]], [[2, 'f7ESF6YBcZ2OE-oMSiQ2LA'], None, [[None, None, 48.86130487660782, 2.29533333364557], [29.27486228942871], [51.32609939575195, 88.81703186035156, 356.7202453613281]], [None, None, [['Port de la Bourdonnais', 'fr']]]], [[2, 'yAt873yc0GqdLKv-tryS8Q'], None, [[None, None, 48.86128833634594, 2.295307404407619], [29.31345558166504], [50.51087188720703, 90.25813293457031, 356.6863098144531]], [None, None, [['Port de la Bourdonnais', 'fr']]]], [[2, 'ng-QSwmV-drWLt9RMd0Ycg'], None, [[None, None, 48.86128834917952, 2.295295152717428], [29.26314163208008], [227.2560424804688, 89.5091323852539, 2.995591878890991]], [None, None, [['Port de la Bourdonnais', 'fr']]]]]], None, None, [[2, [None, None, None, 228.8606872558594]], [1, [None, None, None, 51.46912384033203]]], None, [[17, [2019, 7], None, None, None, 2], [18, [2018, 5], None, None, None, 2], [19, [2017, 8], None, None, None, 2], [20, [2014, 7], None, None, None, 2]], None, None, None, [[[[[None, ['5180951433861232529', '6485737425468686361']], None, ['Port de la Bourdonnais', 'fr']]], [51.46912384033203, 228.8606872558594]]]]], [3, 4, 1, None, None, [None, None, 'launch', [6]], None, [2021, 4]], ['https://www.google.com/local/imagery/report/?cb_client=apiv3\\\\u0026image_key=!1e2!2s5hjX-Yarr0TwxzhXu_FkNg']]]]
raw_md

[[0],
 [[[1],
   [2, '5hjX-Yarr0TwxzhXu_FkNg'],
   [2,
    2,
    [8192, 16384],
    [[[[256, 512]],
      [[512, 1024]],
      [[1024, 2048]],
      [[2048, 4096]],
      [[4096, 8192]],
      [[8192, 16384]]],
     [512, 512]],
    None,
    None,
    None,
    None,
    None,
    '5hjX-Yarr0TwxzhXu_FkNg'],
   [None,
    None,
    [['300 Port de la Bourdonnais', 'fr'],
     ['Paris, \\xc3\\x8ele-de-France', 'en']]],
   [[[['\\xc2\\xa9 2023 Google']]]],
   [[[1],
     [[None, None, 48.86128119507111, 2.295314966617388],
      [29.33272171020508],
      [48.66464614868164, 90.3116226196289, 356.5496826171875],
      None,
      'FR'],
     None,
     [[[[2, '5hjX-Yarr0TwxzhXu_FkNg'],
        None,
        [[None, None, 48.86128119507111, 2.295314966617388],
         [29.33272171020508],
         [48.66464614868164, 90.3116226196289, 356.5496826171875]]],
       [[2, 'P8A5ljT-kyajVoNMihoZ-g'],
        None,
        [[None, None, 48.86134331831732, 2.295420841998263],
         [29.287670

In [46]:
street_name = raw_md[1][0][3][2]
street_name

[['300 Port de la Bourdonnais', 'fr'], ['Paris, \\xc3\\x8ele-de-France', 'en']]

In [41]:
lat, lng = raw_md[1][0][5][0][1][0][2], raw_md[1][0][5][0][1][0][3]
lat, lng

(48.86128119507111, 2.295314966617388)

In [22]:
raw[1][0][6][-1]

[2021, 4]

In [29]:
for pano_info in raw[1][0][5][0][8]:
    raw_pano_info = linked_panos[pano_info[0]]
    print(raw_pano_info)

[[2, 'yWjAwN0FXLMHd1A27JKcOA'], None, [[None, None, 48.86129679955308, 2.295314515323192], [29.22339057922363], [52.39143371582031, 90.56421661376953, 355.9833679199219]], [None, None, [['Port de la Bourdonnais', 'fr']]]]
[[2, 'f7ESF6YBcZ2OE-oMSiQ2LA'], None, [[None, None, 48.86130487660782, 2.29533333364557], [29.27486228942871], [51.32609939575195, 88.81703186035156, 356.7202453613281]], [None, None, [['Port de la Bourdonnais', 'fr']]]]
[[2, 'yAt873yc0GqdLKv-tryS8Q'], None, [[None, None, 48.86128833634594, 2.295307404407619], [29.31345558166504], [50.51087188720703, 90.25813293457031, 356.6863098144531]], [None, None, [['Port de la Bourdonnais', 'fr']]]]
[[2, 'ng-QSwmV-drWLt9RMd0Ycg'], None, [[None, None, 48.86128834917952, 2.295295152717428], [29.26314163208008], [227.2560424804688, 89.5091323852539, 2.995591878890991]], [None, None, [['Port de la Bourdonnais', 'fr']]]]


In [28]:
linked_panos = raw[1][0][5][0][3][0]
linked_panos

[[[2, '5hjX-Yarr0TwxzhXu_FkNg'],
  None,
  [[None, None, 48.86128119507111, 2.295314966617388],
   [29.33272171020508],
   [48.66464614868164, 90.3116226196289, 356.5496826171875]]],
 [[2, 'P8A5ljT-kyajVoNMihoZ-g'],
  None,
  [[None, None, 48.86134331831732, 2.295420841998263],
   [29.28767013549805],
   [48.59828567504883, 89.77833557128906, 356.4241333007812]],
  [None, None, [['Port de la Bourdonnais', 'fr']]]],
 [[2, 'blFcaeu-LX1HwUucDIkHJQ'],
  None,
  [[None, None, 48.86124063275437, 2.295198840434722],
   [29.18351936340332],
   [230.1939849853516, 90.7197036743164, 2.547044515609741]],
  [None, None, [['Port de la Bourdonnais', 'fr']]]],
 [[2, '0h_702dU9Mej3nwwxlNDAQ'],
  None,
  [[None, None, 48.86113720359652, 2.295463031353794],
   [33.45352935791016],
   [44.69409942626953, 86.33415985107422, 0.9682840704917908]]],
 [[2, 'jZy5z48WevCLiwCOoabVmg'],
  None,
  [[None, None, 48.86120294973336, 2.295560545956613],
   [32.62551498413086],
   [45.51588439941406, 86.69369506835938,

In [24]:
raw[1][0][5][0][8]

[[17, [2019, 7], None, None, None, 2],
 [18, [2018, 5], None, None, None, 2],
 [19, [2017, 8], None, None, None, 2],
 [20, [2014, 7], None, None, None, 2]]

In [ ]:
raw_pano_info = linked_panos[pano_info[0]]

In [32]:
import re

In [33]:
_str = """/**/_xdc_._djbvb3 && _xdc_._djbvb3( [[0],[[1],[2,"qhBpqPKAtSw1o7pJI9f4Jg"],[2,2,[8192,16384],[[[[256,512]],[[512,1024]],[[1024,2048]],[[2048,4096]],[[4096,8192]],[[8192,16384]]],[512,512]],null,null,null,null,null,"qhBpqPKAtSw1o7pJI9f4Jg"],[null,null,[["Paris, Île-de-France","en"]]],[[[["© 2023 Google"]]]],[[[1],[[null,null,48.85862296812587,2.292624150524953],[35.79033660888672],[223.7996826171875,88.49066162109375,357.768798828125],null,"FR"],null,[[[[2,"qhBpqPKAtSw1o7pJI9f4Jg"],null,[[null,null,48.85862296812587,2.292624150524953],[35.79033660888672],[223.7996826171875,88.49066162109375,357.768798828125]]],[[2,"wIITz3z19HbKjNKxrwjptA"],null,[[null,null,48.85855458536836,2.292524044758148],[35.60766220092773],[223.7636413574219,88.50564575195312,358.9482116699219]]],[[2,"WRbt--lckZnDj65hPbrKrg"],null,[[null,null,48.85869480774662,2.292728134481407],[35.95196151733398],[223.4369354248047,89.02553558349609,357.9711608886719]]],[[2,"bbpw-bohPRi_UkfSG-vakQ"],null,[[null,null,48.85835384554021,2.292233853890011],[35.10641860961914],[223.8331146240234,88.40332794189453,359.4125366210938]]],[[2,"jcz48RYOB0uiP78T3gguqg"],null,[[null,null,48.8584206037313,2.292330656695505],[35.27642822265625],[223.6757049560547,88.09291076660156,359.5477905273438]]],[[2,"wdDUAqyBgMi1i0WfTURYyw"],null,[[null,null,48.85848751870361,2.292427301451673],[35.43997955322266],[223.7806396484375,88.20585632324219,359.3413696289062]]],[[2,"gakP5B1KI-64R8g5N1Miyg"],null,[[null,null,48.8587669266624,2.292829657250502],[36.08281326293945],[222.5349578857422,88.8310546875,358.903076171875]]],[[2,"Qebw8UEZeMMHndQmOcsFDA"],null,[[null,null,48.85883903166122,2.292928009729465],[36.17997741699219],[222.2659606933594,88.85160827636719,0.1175859421491623]]],[[2,"lgJeE25SbQKPYi6JDGjhTg"],null,[[null,null,48.85891456468903,2.29303169459391],[36.29645919799805],[221.9600219726562,89.62982940673828,359.9290771484375]]],[[2,"l2OhUYnXVg8f7C5UkUQkNA"],null,[[null,null,48.85822687621564,2.292353127940943],[34.75797653198242],[43.05691528320312,90.67064666748047,1.743718266487122]]],[[2,"wDjMoIf8FpeO2jEh0qH3MQ"],null,[[null,null,48.85829171804122,2.292445835232821],[34.89748382568359],[43.03853988647461,90.7611083984375,1.682794570922852]]],[[2,"7pgZYbjv7-2FvOXmSkzW3w"],null,[[null,null,48.85835610476156,2.292538249320226],[35.04335784912109],[43.25143051147461,91.06571197509766,1.710369348526001]]],[[2,"HBCNqjSjwyR2E04u091FRA"],null,[[null,null,48.85842083424689,2.292632747762904],[35.20564270019531],[43.50678634643555,91.22231292724609,1.900143265724182]]],[[2,"fXXItxYouTWmx8WENgGjwg"],null,[[null,null,48.85848594382139,2.292727448851989],[35.36563110351562],[43.25640106201172,90.76031494140625,1.755753636360168]]],[[2,"iatrCq_8pvyVqgx9n-G5UA"],null,[[null,null,48.85854867200035,2.29281827424531],[35.50147247314453],[43.41020584106445,90.86436462402344,1.667503237724304]]],[[2,"IW1TUaL5AffeE92oiZIVHQ"],null,[[null,null,48.85863063011177,2.292938840101654],[35.69191741943359],[44.64569854736328,91.39036560058594,1.735080718994141]]],[[2,"hx3w4Mn4GV6XJh10f-bEOA"],null,[[null,null,48.85870058115733,2.293045071724297],[35.84682083129883],[44.27936553955078,91.08594512939453,1.87370765209198]]],[[2,"5jX7anEpXlo0L8IflXrlqg"],null,[[null,null,48.85876904030723,2.293145186443167],[35.96769332885742],[44.3195915222168,90.92523193359375,1.835105538368225]]],[[2,"8RAUleaFEXtyUp0IVkX_Cw"],null,[[null,null,48.85839330128138,2.292169533957144],[34.50776290893555],[223.3706665039062,91.6453857421875,357.1975708007812]]],[[2,"rOmWH2CFvml3Ij1R0oNl7A"],null,[[null,null,48.85846127317851,2.292267120020071],[34.17092132568359],[222.9272918701172,92.44257354736328,356.8658752441406]]],[[2,"AxTOQJ1mX_zxyIqwSEkljg"],null,[[null,null,48.85852882559302,2.292362616747308],[33.72127532958984],[222.8928527832031,93.15465545654297,357.3728942871094]]],[[2,"EfVHN0nIr7NZpMAst_B82Q"],null,[[null,null,48.85859659282984,2.29245890248295],[33.17473983764648],[223.0916595458984,93.70020294189453,357.4777221679688]]],[[2,"v5y-TyXkJyS1h268tkaClw"],null,[[null,null,48.85866188160223,2.292552332788726],[32.57717132568359],[223.4174041748047,93.73629760742188,357.4144897460938]]],[[2,"8CwydAAD39Ki_gG0GWDh9w"],null,[[null,null,48.85873071277697,2.292653074956381],[31.97720527648926],[223.48681640625,93.79389190673828,357.4516906738281]]],[[2,"KkEbFE1lZu1nLvB5PMPHng"],null,[[null,null,48.8587973329066,2.292749098533773],[31.37393951416016],[223.4241333007812,93.60093688964844,357.4758911132812]]],[[2,"1id9KBmODR9lWmGKSl4rFg"],null,[[null,null,48.85886418312531,2.292845141320828],[30.89320373535156],[223.6441192626953,92.7147216796875,358.7294921875]]],[[2,"eyhlmJxas9mMszVyBKhs0g"],null,[[null,null,48.85893085278817,2.292942540891273],[30.49481391906738],[223.80859375,92.05559539794922,358.7046508789062]]],[[2,"AR7-C5lhekiRRpcoCSgUww"],null,[[null,null,48.85893711492412,2.292138847654247],[28.71578407287598],[49.52152633666992,90.90549468994141,358.7192687988281]]],[[2,"-bHT3X3I2vgvo53PaLgO6A"],null,[[null,null,48.85824484698456,2.292617659720833],[38.77266693115234],[36.77633285522461,86.58132171630859,356.8943176269531],["12343122088187527168"]]],[[2,"2fsPdk1rb77ADLi0XawbjQ"],null,[[null,null,48.858572783736,2.293087724683261],[39.5822639465332],[36.40694808959961,81.31226348876953,357.8325500488281],["12343122088187527168"]]],[[2,"2kGQvu4QaKWnEHv5S6zIng"],null,[[null,null,48.85850292778094,2.292986284728954],[39.41833877563477],[39.78657913208008,84.51804351806641,353.7045288085938],["12343122088187527168"]]],[[2,"7Ow_pmNUr-NhLNqgVsP3RA"],null,[[null,null,48.85836203128781,2.29278286989657],[39.10020065307617],[38.4716796875,82.74539184570312,357.6011047363281],["12343122088187527168"]]],[[2,"DzlYMMbvfuY597tWO8oG4g"],null,[[null,null,48.85831542158429,2.292716412183751],[38.94539642333984],[36.17329788208008,82.48362731933594,355.6266479492188],["12343122088187527168"]]],[[2,"V70pg15EoUZPgXmtBx2czQ"],null,[[null,null,48.85840769584505,2.292854157066671],[39.21155166625977],[43.14750671386719,83.40545654296875,359.9431457519531],["12343122088187527168"]]],[[2,"V7bff3CRNmlirZPWBl5IeQ"],null,[[null,null,48.85865796260084,2.293211460554289],[39.71548461914062],[38.48241806030273,84.48216247558594,356.2003479003906],["12343122088187527168"]]],[[2,"aFx0nVe0GX0oJhz9-Es0Hw"],null,[[null,null,48.85845555753795,2.292921713017449],[39.29069137573242],[39.06051635742188,82.95552825927734,356.0903625488281],["12343122088187527168"]]],[[2,"cB_x0HV7Vd6ieRul18oOgQ"],null,[[null,null,48.85870092892605,2.293270668025542],[39.78107070922852],[40.1765022277832,84.76903533935547,357.4692687988281],["12343122088187527168"]]],[[2,"vGUs095sv2Fqq5gdP476JA"],null,[[null,null,48.85861930445459,2.293150603811256],[39.63104248046875],[43.88811874389648,84.80176544189453,357.4053955078125],["12343122088187527168"]]],[[2,"znDFePmPmf4m02-HhMCSWw"],null,[[null,null,48.85819594521577,2.29255216440302],[38.90715789794922],[33.73110580444336,82.45822143554688,357.1220092773438],["12343122088187527168"]]],[[2,"j6kejYtQVXnS2n7SUZWAqQ"],null,[[null,null,48.85901591455707,2.292373519818995],[33.59711456298828],[321.2079467773438,90.81185150146484,354.6069641113281],["12467628376839094272"]]],[[2,"jrvE7_iipxZkcZi7tgdBxA"],null,[[null,null,48.85901136489447,2.292466001210641],[33.99742889404297],[228.4089508056641,91.90235900878906,356.9912109375],["12467628376839094272"]]],[[2,"w9xKRY8EyCLdl8Ynugip6A"],null,[[null,null,48.85863732472176,2.29264784202071],[35.89577484130859],[223.1858367919922,89.65354156494141,357.4913330078125]]],[[2,"KUaIl3WX4VYSLrJaqK7E8Q"],null,[[null,null,48.85862630592567,2.292649521795095],[35.91595840454102],[223.0176849365234,89.22306060791016,357.8525390625]]],[[2,"3kJFTU5-2I3ktzolNKafRA"],null,[[null,null,48.85862857325506,2.292635911987737],[35.95650100708008],[224.7073211669922,88.95616912841797,357.7524108886719]]],[[2,"Imp2oRBs1ENAylWpTYMIug"],null,[[null,null,48.85863189404581,2.292642776976017],[35.95372009277344],[223.1804351806641,88.12130737304688,357.2275390625]]],[[2,"uTNeHSh9B8tuVgjyXHl0oQ"],null,[[null,null,48.85860833906317,2.292621940414352],[35.88490676879883],[224.1033782958984,89.13259124755859,358.0197143554688]]],[[2,"36tKgquHC5QHXCesauV6sA"],null,[[null,null,48.85863958734956,2.292657705665299],[36.08842468261719],[222.5126037597656,89.09192657470703,357.2492370605469]]],[[2,"OZCr7YGp242yDBCR4Nf4bQ"],null,[[null,null,48.85862248426312,2.292664034059603],[35.99880599975586],[223.5488739013672,89.23651885986328,357.5094909667969]]],[[2,"uP9cki6wOOMQNysDiF4iqQ"],null,[[null,null,48.85862821934109,2.292661498992256],[36.05197525024414],[222.5977020263672,89.26539611816406,358.2568054199219]]],[[2,"ueIQACE-sTITRJg5JEqvSA"],null,[[null,null,48.85861569638099,2.292634443147892],[36.02823638916016],[224.0104217529297,88.96694946289062,357.5411376953125]]],[[2,"8uUSQwmauN-pfyYaZsdg2w"],null,[[null,null,48.85862572722905,2.292620603949179],[36.40463256835938],[223.9107513427734,89.97682952880859,358.0825805664062]]],[[2,"hvMMm_DbxiHIHh7t3w_oKQ"],null,[[null,null,48.8586082216445,2.292623987025971],[36.39229965209961],[223.1982574462891,88.39242553710938,359.1396179199219]]]]],null,null,[[1,[null,null,null,223.2986602783203]],[2,[null,null,null,43.26251983642578]]],null,[[41,[2022,5],null,null,null,2],[42,[2021,4],null,null,null,2],[43,[2020,7],null,null,null,2],[44,[2019,4],null,null,null,2],[45,[2018,5],null,null,null,2],[46,[2017,8],null,null,null,2],[47,[2016,4],null,null,null,2],[48,[2015,7],null,null,null,2],[49,[2014,9],null,null,null,2],[50,[2009,2],null,null,null,2],[51,[2008,5],null,null,null,2]]]],[3,4,1,null,null,[null,null,"launch",[6]],null,[2022,11]],["https://www.google.com/local/imagery/report/?cb_client=apiv3\u0026image_key=!1e2!2sqhBpqPKAtSw1o7pJI9f4Jg"]],[313.4044189453125]] )"""

In [35]:
data = re.findall(r'\[[0-9],"(.+?)"].+?,\[\[null,null,(.+?),(.+?)\]', _str)

In [37]:
data[0][0]

'qhBpqPKAtSw1o7pJI9f4Jg'